In [ ]:

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time
import requests
import lxml
import os

def generate_url(start_date, end_date):  #URL생성함수

    base_url = "https://apis.data.go.kr/1543061/abandonmentPublicSrvc/abandonmentPublic"
    params = (
        f"?bgnde={start_date}"  # Start date
        f"&endde={end_date}"    # End date
        "&upr_cd=6110000"
        "&pageNo=1"
        "&numOfRows=1000"
        "&serviceKey=8rmum9CTW4oN32sj%2Bh1HhAZWQKpLL%2BQzL5v6kXtdpgBMKWPZ0pbM0aXJUUKwAcPDcAplTOgIuZVMa8PMPATRcw%3D%3D"
    )
    return base_url + params


def scrape_abandonment_data(url, download_directory='',  #페이지크롤링함수
                             chromedriver_path=''):

    options = Options()
    prefs = {
        'download.default_directory': download_directory,
        'download.prompt_for_download': False
    }
    options.add_experimental_option('prefs', prefs)

    driver = webdriver.Chrome(service=Service(chromedriver_path), options=options)

    try:

        driver.get(url)
        time.sleep(6)  

        # Parse the page source
        xml_source = driver.page_source
        soup = BeautifulSoup(xml_source, 'xml')
        items = soup.find_all('item')

        # Extract data
        org_nm_list = []
        kind_cd_list = []
        happen_dt_list = []
        processState_list = []
        happenPlace_list = []
        colorCd_list = []
        age_list = []
        weight_list = []
        noticeSdt_list = []
        noticeEdt_list = []
        sexCd_list = []
        specialMark_list = []

        for item in items:
            org_nm = item.find('orgNm').text if item.find('orgNm') else "N/A"
            kind_cd = item.find('kindCd').text if item.find('kindCd') else "N/A"
            happen_dt = item.find('happenDt').text if item.find('happenDt') else "N/A"
            processState = item.find('processState').text if item.find('processState') else "N/A"
            happenPlace = item.find('happenPlace').text if item.find('happenPlace') else "N/A"
            colorCd = item.find('colorCd').text if item.find('colorCd') else "N/A"
            age = item.find('age').text if item.find('age') else "N/A"
            weight = item.find('weight').text if item.find('weight') else "N/A"
            noticeSdt = item.find('noticeSdt').text if item.find('noticeSdt') else "N/A"
            noticeEdt = item.find('noticeEdt').text if item.find('noticeEdt') else "N/A"
            sexCd = item.find('sexCd').text if item.find('sexCd') else "N/A"
            specialMark = item.find('specialMark').text if item.find('specialMark') else "N/A"


            org_nm_list.append(org_nm)
            kind_cd_list.append(kind_cd)
            happen_dt_list.append(happen_dt)
            processState_list.append(processState)
            happenPlace_list.append(happenPlace)
            colorCd_list.append(colorCd)
            age_list.append(age)
            weight_list.append(weight)
            noticeSdt_list.append(noticeSdt)
            noticeEdt_list.append(noticeEdt)
            sexCd_list.append(sexCd)
            specialMark_list.append(specialMark)

        # Create a DataFrame
        data = {
            'orgNm': org_nm_list,
            'kindCd': kind_cd_list,
            'happenDt': happen_dt_list,
            'processState' : processState_list,
            'happenPlace' : happenPlace_list,
            'colorCd' : colorCd_list,
            'age' : age_list,
            'weight' : weight_list,
            'noticeSdt' : noticeSdt_list,
            'noticeEdt' : noticeEdt_list,
            'sexCd' : sexCd_list,
            'specialMark' : specialMark_list,
        }
        df = pd.DataFrame(data)

    finally:
        # Quit the driver
        driver.quit()

    return df


def save_dataframe(df, directory=""):

    if not os.path.exists(directory):
        os.makedirs(directory)

    existing_files = [f for f in os.listdir(directory) if f.endswith('.csv')]
    existing_numbers = [int(os.path.splitext(f)[0]) for f in existing_files if os.path.splitext(f)[0].isdigit()]
    next_number = max(existing_numbers, default=0) + 1
    filename = f"{next_number}.csv"
    filepath = os.path.join(directory, filename)
    df.to_csv(filepath, index=False)

    return filepath